In [1]:
!pip install tenseal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 9.8 MB/s eta 0:00:00


In [2]:
import tenseal as ts

context=ts.context(
        ts.SCHEME_TYPE.CKKS,
        poly_modulus_degree=8192,
        coeff_mod_bit_sizes=[60, 40, 40, 60]
    )
context.generate_galois_keys()
context.global_scale=2**40

In [3]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [4]:
X, y = make_classification(n_samples=1000, n_features=10, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
enc_X_train=[]
for i in X_train:
  p=ts.ckks_vector(context,i)
  enc_X_train.append(p)

In [6]:
enc_y_train=[]
for i in y_train:
  p=ts.ckks_vector(context,[i])
  enc_y_train.append(p)

In [7]:
def sigmoid(z):
  z=z.decrypt()
  z=round(z[0],7)
  x=1/(1+np.exp(-z))
  x=round(x,7)
  x=ts.ckks_vector(context,[x])
  return x

In [8]:
def relinearlizing_vector(vec):
  vec_2=[]
  vec=vec.decrypt()
  for i in vec:
    p=round(i,7)
    vec_2.append(p)
  vec_2=ts.ckks_vector(context,vec_2)
  return vec_2

In [9]:
def relinearlizing_scalar(sca):
  sca=sca.decrypt()
  sca=round(sca[0],7)
  sca=ts.ckks_vector(context,[sca])
  return sca

In [10]:
n_samples,n_attributes=X_train.shape
n_samples,n_attributes

(800, 10)

In [11]:
weights=np.zeros(n_attributes)
bias=0
lr=0.01
weights,bias

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 0)

In [12]:
weights=ts.ckks_vector(context,weights)
bias=ts.ckks_vector(context,[bias])

In [13]:
ar=np.arange(n_samples)
for epoch in range(5):
  np.random.shuffle(ar)
  for i in ar:
    xi=enc_X_train[i]
    yi=enc_y_train[i]
    lin_op=xi.dot(weights)
    lin_op=lin_op+bias
    y_pred=sigmoid(lin_op)
    db=y_pred-yi
    dw=db*xi
    dw=relinearlizing_vector(dw)
    db=relinearlizing_scalar(db)
    weights=relinearlizing_vector(weights)
    bias=relinearlizing_scalar(bias)
    weights=weights-(lr*dw)
    bias=bias-(lr*db)
    print(f'epoch={epoch}, {i} loop is executed')

epoch=0, 231 loop is executed
epoch=0, 503 loop is executed
epoch=0, 293 loop is executed
epoch=0, 478 loop is executed
epoch=0, 427 loop is executed
epoch=0, 363 loop is executed
epoch=0, 132 loop is executed
epoch=0, 403 loop is executed
epoch=0, 518 loop is executed
epoch=0, 657 loop is executed
epoch=0, 144 loop is executed
epoch=0, 330 loop is executed
epoch=0, 586 loop is executed
epoch=0, 676 loop is executed
epoch=0, 169 loop is executed
epoch=0, 76 loop is executed
epoch=0, 476 loop is executed
epoch=0, 472 loop is executed
epoch=0, 268 loop is executed
epoch=0, 383 loop is executed
epoch=0, 223 loop is executed
epoch=0, 102 loop is executed
epoch=0, 165 loop is executed
epoch=0, 398 loop is executed
epoch=0, 232 loop is executed
epoch=0, 565 loop is executed
epoch=0, 6 loop is executed
epoch=0, 10 loop is executed
epoch=0, 796 loop is executed
epoch=0, 18 loop is executed
epoch=0, 592 loop is executed
epoch=0, 278 loop is executed
epoch=0, 52 loop is executed
epoch=0, 529 loo

In [14]:
weights.decrypt()

[-0.4522849672223969,
 0.13480163503933185,
 -0.7352126765393106,
 0.05300667998299502,
 -0.020780553774704558,
 -0.20562655511200856,
 1.5900527707108356,
 -0.014560849933279177,
 -0.5963197522812282,
 0.09933947865799837]

In [15]:
bias.decrypt()

[0.12326753791764351]

In [16]:
from sklearn.linear_model import LogisticRegression

In [17]:
model = LogisticRegression(max_iter=1000, solver='saga')
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, solver='saga')

In [18]:
weights_sklearn = model.coef_[0]
bias_sklearn = model.intercept_[0]

In [19]:
print("Weights from scikit-learn:", weights_sklearn)
print("Bias from scikit-learn:", bias_sklearn)

Weights from scikit-learn: [-0.45220344  0.16123384 -0.74511296  0.0554711  -0.04317329 -0.22874173
  1.62041827 -0.00332163 -0.62900151  0.10251058]
Bias from scikit-learn: 0.13591967516833706


In [20]:
print("\nComparison of Weights and Bias:")
print("Weights difference:", np.abs(weights.decrypt() - weights_sklearn))
print("Bias difference:", np.abs(bias.decrypt() - bias_sklearn))


Comparison of Weights and Bias:
Weights difference: [8.15223869e-05 2.64322084e-02 9.90028309e-03 2.46442355e-03
 2.23927408e-02 2.31151711e-02 3.03654966e-02 1.12392248e-02
 3.26817560e-02 3.17110337e-03]
Bias difference: [0.01265214]
